In [ ]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_64x64_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 10)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'DCT_64_Depth10/5 Projections'

DCT_10_proj5_weights, DCT_10_proj5_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj5', display=True)

DCT_20_proj5_weights, DCT_20_proj5_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj5', display=True)

DCT_30_proj5_weights, DCT_30_proj5_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj5', display=True)

DCT_40_proj5_weights, DCT_40_proj5_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, DCT_10_proj5_weights, DCT_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, DCT_20_proj5_weights, DCT_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, DCT_30_proj5_weights, DCT_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, DCT_40_proj5_weights, DCT_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.10641706721650229
Average test loss: 0.00481374772472514
Epoch 2/300
Average training loss: 0.027218736234638427
Average test loss: 0.004473803512338135
Epoch 3/300
Average training loss: 0.02386380257540279
Average test loss: 0.004489010292208857
Epoch 4/300
Average training loss: 0.022653225814302763
Average test loss: 0.004118203583276934
Epoch 5/300
Average training loss: 0.022027290264765423
Average test loss: 0.004086458264332678
Epoch 6/300
Average training loss: 0.021621337282988762
Average test loss: 0.0040368094945119485
Epoch 7/300
Average training loss: 0.021327080173624887
Average test loss: 0.004005931794229481
Epoch 8/300
Average training loss: 0.021025365738405122
Average test loss: 0.003984615670517087
Epoch 9/300
Average training loss: 0.020858050568236246
Average test loss: 0.003989629129775696
Epoch 10/300
Average training loss: 0.020678447772231368
Average test loss: 0.003937037494033575


In [ ]:
DCT_10_proj5_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj5_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj5_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj5_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj5_psnr = average_PSNR(DCT_10_proj5_model, DCT_10_proj5_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj5_psnr = average_PSNR(DCT_20_proj5_model, DCT_20_proj5_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj5_psnr = average_PSNR(DCT_30_proj5_model, DCT_30_proj5_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj5_psnr = average_PSNR(DCT_40_proj5_model, DCT_40_proj5_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj5_psnr, f)

In [ ]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'DCT_64_Depth10/15 Projections'

DCT_10_proj15_weights, DCT_10_proj15_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj15', display=True)

DCT_20_proj15_weights, DCT_20_proj15_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj15', display=True)

DCT_30_proj15_weights, DCT_30_proj15_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj15', display=True)

DCT_40_proj15_weights, DCT_40_proj15_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, DCT_10_proj15_weights, DCT_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, DCT_20_proj15_weights, DCT_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, DCT_30_proj15_weights, DCT_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, DCT_40_proj15_weights, DCT_40_proj15_hist, '40% Sampling')

In [ ]:
DCT_10_proj15_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj15_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj15_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj15_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj15_psnr = average_PSNR(DCT_10_proj15_model, DCT_10_proj15_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj15_psnr = average_PSNR(DCT_20_proj15_model, DCT_20_proj15_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj15_psnr = average_PSNR(DCT_30_proj15_model, DCT_30_proj15_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj15_psnr = average_PSNR(DCT_40_proj15_model, DCT_40_proj15_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj15_psnr, f)

In [7]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'DCT_64_Depth10/30 Projections'

DCT_10_proj30_weights, DCT_10_proj30_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj30', display=True)

DCT_20_proj30_weights, DCT_20_proj30_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj30', display=True)

DCT_30_proj30_weights, DCT_30_proj30_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj30', display=True)

DCT_40_proj30_weights, DCT_40_proj30_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, DCT_10_proj30_weights, DCT_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, DCT_20_proj30_weights, DCT_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, DCT_30_proj30_weights, DCT_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, DCT_40_proj30_weights, DCT_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 31.96649715338813
Average test loss: 0.047179391310446794
Epoch 2/300
Average training loss: 12.61924392361111
Average test loss: 0.13452022591895527
Epoch 3/300
Average training loss: 9.361921060350205
Average test loss: 0.004632598211367925
Epoch 4/300
Average training loss: 7.393131803724501
Average test loss: 0.004556335165682766
Epoch 5/300
Average training loss: 5.088184977001614
Average test loss: 0.005590287084173825
Epoch 6/300
Average training loss: 3.200823096593221
Average test loss: 0.008661017233712806
Epoch 7/300
Average training loss: 2.6987247053782144
Average test loss: 0.03586840388696227
Epoch 8/300
Average training loss: 2.063296583175659
Average test loss: 0.10739847055988179
Epoch 9/300
Average training loss: 1.7044853920406766
Average test loss: 0.26803768113172716
Epoch 10/300
Average training loss: 1.417381526099311
Average test loss: 0.004375744263538056
Epoch 11/300
Average training 

In [8]:
DCT_10_proj30_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj30_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj30_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj30_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj30_psnr = average_PSNR(DCT_10_proj30_model, DCT_10_proj30_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj30_psnr = average_PSNR(DCT_20_proj30_model, DCT_20_proj30_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj30_psnr = average_PSNR(DCT_30_proj30_model, DCT_30_proj30_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj30_psnr = average_PSNR(DCT_40_proj30_model, DCT_40_proj30_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.39
Average PSNR for Projection Layer 1 across 2500 images: 27.84
Average PSNR for Projection Layer 2 across 2500 images: 27.94
Average PSNR for Projection Layer 3 across 2500 images: 27.98
Average PSNR for Projection Layer 4 across 2500 images: 28.10
Average PSNR for Projection Layer 5 across 2500 images: 28.18
Average PSNR for Projection Layer 6 across 2500 images: 28.17
Average PSNR for Projection Layer 7 across 2500 images: 28.19
Average PSNR for Projection Layer 8 across 2500 images: 28.23
Average PSNR for Projection Layer 9 across 2500 images: 27.96
Average PSNR for Projection Layer 10 across 2500 images: 28.27
Average PSNR for Projection Layer 11 across 2500 images: 28.33
Average PSNR for Projection Layer 12 across 2500 images: 28.31
Average PSNR for Projection Layer 13 across 2500 images: 28.36
Average PSNR for Projection Layer 14 across 2500 images: 28.39
Average PSNR for Projection Layer 15 across 2500 images: 28.40
Av